## Урок 6 

In [707]:
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import mean_squared_error,r2_score 
import random
from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix
%matplotlib inline
import matplotlib.pylab as plt

### Задание 1

Взять любой набор данных для бинарной классификации.

In [708]:
df = pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.csv")
df.head(3)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes


In [709]:
df.shape

(7043, 21)

### Задание 2

Сделать feature engineering.

In [710]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [711]:
for col in df.columns:
    print(col, ':', df[col].unique(), 'Значений -', len(df[col].unique()))

customerID : ['7590-VHVEG' '5575-GNVDE' '3668-QPYBK' ... '4801-JZAZL' '8361-LTMKD'
 '3186-AJIEK'] Значений - 7043
gender : ['Female' 'Male'] Значений - 2
SeniorCitizen : [0 1] Значений - 2
Partner : ['Yes' 'No'] Значений - 2
Dependents : ['No' 'Yes'] Значений - 2
tenure : [ 1 34  2 45  8 22 10 28 62 13 16 58 49 25 69 52 71 21 12 30 47 72 17 27
  5 46 11 70 63 43 15 60 18 66  9  3 31 50 64 56  7 42 35 48 29 65 38 68
 32 55 37 36 41  6  4 33 67 23 57 61 14 20 53 40 59 24 44 19 54 51 26  0
 39] Значений - 73
PhoneService : ['No' 'Yes'] Значений - 2
MultipleLines : ['No phone service' 'No' 'Yes'] Значений - 3
InternetService : ['DSL' 'Fiber optic' 'No'] Значений - 3
OnlineSecurity : ['No' 'Yes' 'No internet service'] Значений - 3
OnlineBackup : ['Yes' 'No' 'No internet service'] Значений - 3
DeviceProtection : ['No' 'Yes' 'No internet service'] Значений - 3
TechSupport : ['No' 'Yes' 'No internet service'] Значений - 3
StreamingTV : ['No' 'Yes' 'No internet service'] Значений - 3
StreamingM

In [712]:
df = pd.get_dummies(df, columns = ['InternetService', 'Contract', 'PaymentMethod'])

In [713]:
df.gender = np.where((df.gender == 'Female'), 1, 0)

In [714]:
yes_no = ['Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 
          'TechSupport', 'StreamingTV', 'StreamingMovies', 'PaperlessBilling', 'Churn', 'TotalCharges']

In [715]:
df.loc[(df['TotalCharges'] == ' '), 'TotalCharges'] = 0

In [716]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'])

In [717]:
for col in df[yes_no]:
    df[col] = np.where((df[col] == "Yes"), 1, 0)        

In [718]:
df.head(3)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,...,InternetService_DSL,InternetService_Fiber optic,InternetService_No,Contract_Month-to-month,Contract_One year,Contract_Two year,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,7590-VHVEG,1,0,1,0,1,0,0,0,1,...,1,0,0,1,0,0,0,0,1,0
1,5575-GNVDE,0,0,0,0,34,1,0,1,0,...,1,0,0,0,1,0,0,0,0,1
2,3668-QPYBK,0,0,0,0,2,1,0,1,1,...,1,0,0,1,0,0,0,0,0,1


In [719]:
df['customerID'] = df['customerID'].str[:4]

In [720]:
df['customerID'] = pd.to_numeric(df['customerID'])

In [721]:
for col in df.columns:
    print(col, ':', df[col].unique(), 'Значений -', len(df[col].unique()))

customerID : [7590 5575 3668 ... 7750 2569 6840] Значений - 5084
gender : [1 0] Значений - 2
SeniorCitizen : [0 1] Значений - 2
Partner : [1 0] Значений - 2
Dependents : [0 1] Значений - 2
tenure : [ 1 34  2 45  8 22 10 28 62 13 16 58 49 25 69 52 71 21 12 30 47 72 17 27
  5 46 11 70 63 43 15 60 18 66  9  3 31 50 64 56  7 42 35 48 29 65 38 68
 32 55 37 36 41  6  4 33 67 23 57 61 14 20 53 40 59 24 44 19 54 51 26  0
 39] Значений - 73
PhoneService : [0 1] Значений - 2
MultipleLines : [0 1] Значений - 2
OnlineSecurity : [0 1] Значений - 2
OnlineBackup : [1 0] Значений - 2
DeviceProtection : [0 1] Значений - 2
TechSupport : [0 1] Значений - 2
StreamingTV : [0 1] Значений - 2
StreamingMovies : [0 1] Значений - 2
PaperlessBilling : [1 0] Значений - 2
MonthlyCharges : [29.85 56.95 53.85 ... 63.1  44.2  78.7 ] Значений - 1585
TotalCharges : [0] Значений - 1
Churn : [0 1] Значений - 2
InternetService_DSL : [1 0] Значений - 2
InternetService_Fiber optic : [0 1] Значений - 2
InternetService_No : [

### Задание 3

Обучить любой классификатор (какой вам нравится).

In [722]:
X = df.drop(['Churn'], axis = 1)

In [723]:
y = df['Churn']

In [724]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    train_size=0.7, 
                                                    random_state=42)

In [725]:
params = {'n_estimators':200,
          'max_depth':12,
          'criterion':'mse',
          'learning_rate':0.03,
          'min_samples_leaf':16,
          'min_samples_split':16
          }

In [726]:
gbr = GradientBoostingClassifier(**params)
gbr.fit(X_train,y_train)

GradientBoostingClassifier(criterion='mse', learning_rate=0.03, max_depth=12,
                           min_samples_leaf=16, min_samples_split=16,
                           n_estimators=200)

In [727]:
y_pred = gbr.predict(X_test)

In [728]:
comparison = []

In [729]:
precision, recall, thresholds = precision_recall_curve(y_test, y_pred)

fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)

if ['GradientBoostingClassifier', precision[ix], recall[ix], thresholds[ix], fscore[ix]] not in comparison:
    comparison.append(['RGradientBoostingClassifier', precision[ix], recall[ix], thresholds[ix], fscore[ix]])
    
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

Best Threshold=1.000000, F-Score=0.568, Precision=0.638, Recall=0.512


In [730]:
train_accuracy_score=gbr.score(X_train,y_train)
print(train_accuracy_score)

test_accuracy_score=gbr.score(X_test,y_test)
print(test_accuracy_score)

0.9647058823529412
0.7884524372929484


### Задание 4

Далее разделить ваш набор данных на два множества: P (positives) и U (unlabeled). Причем брать нужно не все положительные (класс 1) примеры, а только лишь часть.

Всего позитивов у нас:

In [731]:
df['Churn'].value_counts()

0    5174
1    1869
Name: Churn, dtype: int64

In [734]:
data_P = df.loc[(df['Churn'] == 1) & (df['customerID'] % 4== 0)]

<font color = 'red'> А как можно было по-нормальному поделить базу на позитивы?

In [737]:
data_P.shape

(451, 28)

In [738]:
data_U = pd.concat([df, data_P]).drop_duplicates(keep=False)

### Задание 5

Применить random negative sampling для построения классификатора в новых условиях.

In [739]:
NP = data_P.shape[0]
NU = data_U.shape[0]

In [740]:
y_test = data_U['Churn']

In [741]:
y_test = np.array(y_test)

In [742]:
data_U = np.array(data_U)

In [743]:
data_P = np.array(data_P)

In [744]:
NP = data_P.shape[0]
NU = data_U.shape[0]

T = 1000
K = NP
train_label = np.zeros(shape=(NP+K,))
train_label[:NP] = 1.0
n_oob = np.zeros(shape=(NU,))
f_oob = np.zeros(shape=(NU, 2))
for i in range(T):
    # Bootstrap resample
    bootstrap_sample = np.random.choice(np.arange(NU), replace=True, size=K)
    # Positive set + bootstrapped unlabeled set
    data_bootstrap = np.concatenate((data_P, data_U[bootstrap_sample, :]), axis=0)
    # Train model
    model = DecisionTreeClassifier(max_depth=None, max_features=None, 
                                   criterion='gini', class_weight='balanced')
    model.fit(data_bootstrap, train_label)
    # Index for the out of the bag (oob) samples
    idx_oob = sorted(set(range(NU)) - set(np.unique(bootstrap_sample)))
    # Transductive learning of oob samples
    f_oob[idx_oob] += model.predict_proba(data_U[idx_oob])
    n_oob[idx_oob] += 1
predict_proba = f_oob[:, 1]/n_oob

In [745]:
precision, recall, thresholds = precision_recall_curve(y_test, predict_proba)

fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)

if ['Random_negative_sampling ', precision[ix], recall[ix], thresholds[ix], fscore[ix]] not in comparison:
    comparison.append(['Random_negative_sampling ', precision[ix], recall[ix], thresholds[ix], fscore[ix]])

print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

Best Threshold=0.377682, F-Score=1.000, Precision=1.000, Recall=1.000


### Задание 6

Cравнить качество с решением из пункта 4 (построить отчет - таблицу метрик).

In [746]:
df_fin = pd.DataFrame(comparison, columns =['model', 'precision', 'recall', 'thresholds', 'fscore'])

In [747]:
df_fin

,model,precision,recall,thresholds,fscore
0,RGradientBoostingClassifier,0.637744,0.512195,1.000000,0.568116
1,Random_negative_sampling,1.000000,1.000000,0.377682,1.000000


<font color = 'red'> Я где-то туплю с расчетом показателей в варианте Random_negative_sampling. Но не понимаю, что нужно исправить.

### Задание 7

Поэкспериментировать с долей P на шаге 5 (как будет меняться качество модели при уменьшении/увеличении размера P).

In [748]:
# Best Threshold=0.327766, F-Score=1.000, Precision=1.000, Recall=1.000 - 311 записей в P
# Best Threshold=0.377682, F-Score=1.000, Precision=1.000, Recall=1.000 - 451 запись в Р